In [1]:
!nvidia-smi

Fri Nov 13 12:00:16 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    24W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
ROOT_DIR = "/content/drive/My Drive/"
os.chdir(os.path.join(ROOT_DIR, "medico_task_2_submission"))

Mounted at /content/drive


In [3]:
!pip install torch torchvision
# install dependencies: (use cu100 because colab is on CUDA 10.0)
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

#torch.__version__
#!gcc --version
# opencv is pre-installed on colab


# install detectron2:
#!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/index.html

     |████████████████████████████████| 276kB 12.5MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44075 sha256=7df8bffb98e4306de0899807236d25219d1e6e5e5e7b982a865463fb518ab340
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-d6ya79c0
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-d6ya79c0
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-linux_x86_64.whl size=266457 sha256=95097a05209afddad618c9307f6d949b3b0fa3656f2b2d5cfdb7b1ae938a5462
  Stored in directory: /tmp/pip-ephem-wheel-cache-odadkzis/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools
  Found existing instal

In [4]:
#install RESNEST REPO
!pip install 'git+https://github.com/zhanghang1989/detectron2-ResNeSt'

  Cloning https://github.com/zhanghang1989/detectron2-ResNeSt to /tmp/pip-req-build-9yoity67
  Running command git clone -q https://github.com/zhanghang1989/detectron2-ResNeSt /tmp/pip-req-build-9yoity67
  Created wheel for detectron2: filename=detectron2-0.1.1-cp36-cp36m-linux_x86_64.whl size=4735530 sha256=8a327d84403e6a1f6db601af266acc426e507e210acccd703acaa659fbe9b274
  Stored in directory: /tmp/pip-ephem-wheel-cache-_xmomn7u/wheels/ab/4b/13/cfbbb4bc6a8309d75353f018ff373a5c1eecfb2e6526711f95
  Created wheel for fvcore: filename=fvcore-0.1.dev200407-cp36-none-any.whl size=38762 sha256=6075d10b079251502f852febc31e4511067a8e9b347d906db12641a5301ab1ef
  Stored in directory: /root/.cache/pip/wheels/0a/78/7a/85cd228669b8c0db682b13f7f34589b7351281111e6002a7cc
Successfully built detectron2 fvcore


In [5]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import torch, torchvision

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [6]:
!pip install tifffile

# Predict with model on test data



In [7]:
from detectron2.data.datasets import register_coco_instances
#register_coco_instances("val", {}, "datasets/polyps2.json", "datasets/polyps2")
register_coco_instances("train", {}, "datasets/kvasir_train_corrected.json", "datasets/Kvasir-SEG")

In [8]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer, SimpleTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset, SemSegEvaluator
from detectron2.data import build_detection_test_loader

experiment = 'mask_cascade_rcnn_ResNeSt_50_FPN_syncBN_1x'

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/" + experiment + ".yaml"))
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 10000    # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon)

cfg.OUTPUT_DIR = 'experiments_gpu_server/' + experiment

cfg.MODEL.WEIGHTS = os.path.join('experiments_gpu_server/' + experiment, "model_0002799.pth")

cfg.DATASETS.TRAIN = ("train",)

trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
# another equivalent way is to use trainer.test

[11/13 12:05:43 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(
      256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
      (norm): NaiveSyncBatchNorm(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (fpn_output2): Conv2d(
      256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (norm): NaiveSyncBatchNorm(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (fpn_lateral3): Conv2d(
      512, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
      (norm): NaiveSyncBatchNorm(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (fpn_output3): Conv2d(
      256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (norm): NaiveSyncBatchNorm(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (fpn_lateral4): Conv2d(
      1024, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
      (

In [9]:
path = 'datasets/polyps2/'
val_data = os.listdir(path)
_ = sorted(val_data)

In [10]:
import tifffile
from detectron2.utils.visualizer import ColorMode
predictor = DefaultPredictor(cfg)

Dict = {0:'polyp'} 

output_folder = '/prediction_masks_2799/'
os.makedirs(cfg.OUTPUT_DIR+ output_folder, exist_ok=True)
for img_path in val_data: 
    if img_path.endswith('.jpg'):   
        print(img_path)
        im = cv2.imread(path + img_path)
        outputs = predictor(im)
        txt_fullpath = cfg.OUTPUT_DIR + output_folder + os.path.splitext(img_path)[0]+'.txt'  # eg. 0001

        with open(txt_fullpath, 'w+') as file:
            full_text = ''
            #for i, _ in enumerate(outputs['instances'].get('pred_boxes').tensor):
            x_min = str(int(round(outputs['instances'].get('pred_boxes').tensor[0][0].item())))
            y_min = str(int(round(outputs['instances'].get('pred_boxes').tensor[0][1].item())))
            x_max = str(int(round(outputs['instances'].get('pred_boxes').tensor[0][2].item())))
            y_max = str(int(round(outputs['instances'].get('pred_boxes').tensor[0][3].item())))
            score = str(round(outputs['instances'].get('scores')[0].item(),2))
            object_class = int(outputs['instances'].get('pred_classes')[0].item())
            string = Dict[object_class] + " " + score + " " + x_min + " " + y_min + " " + x_max + " " + y_max + '\n'
            full_text = full_text + string

            
            new_shape = list(im.shape)
            new_shape[2] = 1
            new_shape[0], new_shape[2] = new_shape[2], new_shape[0]
            new_shape[1], new_shape[2] = new_shape[2], new_shape[1]
            out = np.zeros(new_shape)
            #for i, _ in enumerate(outputs["instances"].get('pred_masks')):
                
            object_class_seg = int(outputs['instances'].get('pred_classes')[0].item())
            out[object_class_seg,:,:] = outputs["instances"].get('pred_masks')[0].to("cpu").numpy()*255
            out = out.astype(int)
            out = np.uint8(out)
            tifffile.imsave(cfg.OUTPUT_DIR + output_folder +img_path.split('.')[0]+'.tif', out)
            full_text = full_text[:-2]
            file.write(full_text)

ckd8rkeea00063b5segvd2va3.jpg


/usr/local/lib/python3.6/dist-packages/detectron2/modeling/roi_heads/fast_rcnn.py:107: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  filter_inds = filter_mask.nonzero()


ckd8rln2j00073b5s5t3jb0ef.jpg
ckd8rmjd700083b5sk7ajpzf6.jpg
ckd8rnczu00093b5s3nlrjr2p.jpg
ckd8sr33b00003b5sl6899a8l.jpg
ckd8srbmv00013b5sliek3ejo.jpg
ckd8srxq500023b5ssndncx7o.jpg
ckd8ssoan00033b5sdjgxl66l.jpg
ckd8stdnx00043b5sxun8ewkv.jpg
ckd8strm900053b5srwqcah9s.jpg
ckd8sun6y00063b5swzdd3fd8.jpg
ckd8sv8ds00073b5scw72l1dy.jpg
ckd8swai400083b5sn1693d25.jpg
ckd8swq2100093b5s1zjkgj4b.jpg
ckd8sxdyn000a3b5sclphcakf.jpg
ckd8sxr2u000b3b5ssko8d40l.jpg
ckd8sy9sg000c3b5sgrphj4u8.jpg
ckd8synvm000d3b5shaneypja.jpg
ckd8sz9ut000e3b5s6vs09c0c.jpg
ckd8tnm3g000h3b5s1rxnjsq3.jpg
ckd8tmpp3000g3b5scbgr51m8.jpg
ckd8toa0f000i3b5swbpasudq.jpg
ckd8tq1xr000j3b5s1kn6vx34.jpg
ckd8tspte00003b5seb3ryptl.jpg
ckd8ttjwc00013b5s33m1jgq2.jpg
ckd8ttzeo00023b5sfro24eom.jpg
ckd8tucza00033b5sm65huh8r.jpg
ckd8tuxn000043b5s8bwtvcom.jpg
ckd8tvw6j00053b5swagqipc1.jpg
ckd8twk1a00063b5slocqudcl.jpg
ckd8tx25100073b5sn7cjx32x.jpg
ckd8txnqo00083b5spbqvehzd.jpg
ckd8tyc3z00093b5s2p9b0ges.jpg
ckd8u03ax000b3b5s6wlo2oa1.jpg
ckd8u18d70